# 3D Example

In [1]:
from ngsolve import *
from netgen.occ import *
from netgen.meshing import MeshingParameters
from netgen.webgui import Draw

iron = Box((0,0,0),(1,1,1))
iron.name = "iron"
iron.maxh = 0.2
internal_domain = Sphere(Pnt(0,0,0.0), r=2)*Box((0,0,0), (5,5,5))
internal_domain.name = "air"
internal_domain.maxh = 1.0
internal_domain.faces.Min(Z).name = "gnd"
internal_domain.faces[0].name = "outer"

external_domain=Sphere(Pnt(3,0,0.0), r=2)*Box((3,0,0), (3+5,5,5))
external_domain.name = "air"
external_domain.maxh = 1.0
external_domain.faces.Min(Z).name = "gnd"
external_domain.faces[0].name = "outer"

external_domain.faces[0].Identify(internal_domain.faces[0], "id")
domain = Glue([iron, internal_domain, external_domain])
#Draw(domain)

geo = OCCGeometry(domain, dim=3)
mp = MeshingParameters(maxh=20.0, segmentsperedge=1, grading=0.7)

from ngsolve.webgui import Draw
mesh = Mesh(geo.GenerateMesh(mp)).Curve(2)

print(mesh.GetMaterials())
print(mesh.GetBoundaries())

plist = []
for pair in mesh.ngmesh.GetIdentifications():
    plist += list(mesh.ngmesh[pair[0]].p)
    plist += list(mesh.ngmesh[pair[1]].p)
#   print([mesh.vertices[pair[0]-1].point, mesh.vertices[pair[1]-1].point])
Draw(mesh, objects=[{"type" : "lines", "position" : plist, "name": "identification", "color": "red"}]);

fes = Periodic(H1(mesh, order=1, dirichlet='gnd'))
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes) 
a += grad(u)*grad(v)*dx
a.Assemble()

f = LinearForm(fes)
f += v*z*dx(definedon=mesh.Materials("iron"))
f.Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu, mesh);


('iron', 'default', 'default')
('default', 'default', 'default', 'default', 'gnd', 'default', 'outer', 'default', 'gnd', 'default', 'outer', 'default', 'gnd', 'default')


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…